# Sistemas Basados en Reglas usando Experta

En primer lugar, debemos instalar la librería "experta" (fork de pyknow).

La documentación oficial de la librería se puede encontrar en:
https://experta.readthedocs.io/en/latest/index.html

Esta librería está basada en el lenguaje de programación CLIPS, escrito en C.

In [ ]:
%pip install experta

  Preparing metadata (setup.py) ... done
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=7e04d6e691ce0e6741f2041cf0703e4373f3f3e4ccc93e19daef9ae8fc3cd988
  Stored in directory: /root/.cache/pip/wheels/ff/80/86/2d516a3c26397f67adaa2c848879d4a6359d90a60546ce4e03
Successfully built frozendict
  Attempting uninstall: frozendict
    Found existing installation: frozendict 2.4.0
    Uninstalling frozendict-2.4.0:
      Successfully uninstalled frozendict-2.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.37 requires frozendict>=2.3.4, but you have frozendict 1.2 which is incompatible.


# Hechos (Facts)

En esta sección aprenderemos a crear Hechos de nuestra BH (base de hechos).
Para ello, crearemos objetos de la clase Fact.

In [ ]:
from experta import *

In [ ]:
f = Fact(a=1, b=2)
print(f['a'], f['b'])
f

1 2


Fact(a=1, b=2)

In [ ]:
class Alert(Fact):
  """The alert level."""
  pass

class Status(Fact):
  """The system status."""
  pass

f1 = Alert('red')
f2 = Status('critical')

In [ ]:
f1

Alert('red')

## Reglas (rules)

Las reglas tienen dos partes:
- Parte izquierda (LHS) condiciones para ejecutar la regla.
- Parte derecha (RHS) acciones que se ejecutan cuando se cumple la parte LHS.

In [ ]:
# En nuestro motor de inferencia introducimos una regla match_aprobado
class Notas(KnowledgeEngine):
  @Rule(Fact(nota='aprobado'))
  def match_aprobado(self):
    print("Aprobado!")

## Funcionamiento del motor de inferencia

1. Se instancia el motor (KnowledgeEngine)
2. Se llama al método reset


> *   Se declara el hecho InitialFact
> *   Se declaran todos los métodos del método decorado por @DefFacts


3. Se llama al método run para comenzar la ejecución

In [ ]:
engine = Notas()
engine.reset()
print("Antes: ", engine.facts)
engine.declare(Fact(nota='aprobado'))
print("Después: ", engine.facts)
engine.run()

Antes:  <f-0>: InitialFact()
Después:  <f-0>: InitialFact()
<f-1>: Fact(nota='aprobado')
Aprobado!


In [ ]:
# Para obtener la base de hechos
engine.facts

FactList([(0, InitialFact()), (1, Fact(nota='aprobado'))])

## DefFacts

Son hechos que deben estar presentes al iniciar el sistema basados en reglas para que este 'arranque'.

In [ ]:
class Test(KnowledgeEngine):
  @DefFacts()
  def start_data(self):
    yield Fact(color='blue')
    yield Fact(size='large')

  @Rule(Fact(size='small'))
  def now_is_small(self):
    print('Now is small!')

In [ ]:
test = Test()
test.reset()
test.declare(Fact(size='small'))
test.run()

Now is small!


In [ ]:
test.facts

FactList([(0, InitialFact()),
          (1, Fact(color='blue')),
          (2, Fact(size='large')),
          (3, Fact(size='small'))])

## DefFacts vs declare

Ambos son usados para introducir hechos en la Base de Hechos, pero la diferencia es la siguiente:

- declare añade el hecho a la memoria directamente
- DefFacts se usa para declarar generadores, que serán creados por la llamada al método reset. A partir de esta llamada, todos los hechos generados en DefFacts serán añadidos a memoria usando declare

## Ejemplo de uso de reglas con Rule y DefFacts


In [ ]:
from experta import *

EDAD_LIMITE = 18

class Registro(KnowledgeEngine):
  @DefFacts()
  def inicializacion(self):
    yield Fact(accion="inicio")

  @Rule(Fact(accion='inicio'),
  NOT(Fact(edad=W())))
  def preguntar_edad(self):
    self.declare(Fact(edad=int(input("¿Cuál es tu edad? "))))

  @Rule(Fact(accion='inicio'),
  NOT(Fact(nombre=W())))
  def preguntar_nombre(self):
    self.declare(Fact(nombre=input("¿Cuál es tu nombre? ")))

  @Rule(Fact(accion='inicio'),
  Fact(nombre=MATCH.nombre),
  Fact(edad=MATCH.edad & GE(EDAD_LIMITE)))
  def bienvenida(self, nombre, edad):
    print("¡Bienvenido %s! Tu edad es %s" % (nombre, edad))

  @Rule(Fact(accion='inicio'),
  Fact(nombre=MATCH.nombre),
  Fact(edad=MATCH.edad & LT(EDAD_LIMITE)))
  def no_bienvenida(self, nombre, edad):
    print("%s, tu edad es %s, no puedes hacer login" % (nombre, edad))

engine = Registro()
engine.reset() # Prepare the engine for the execution.
engine.run() # Run it!

¿Cuál es tu edad? 15
¿Cuál es tu nombre? Paco
Paco, tu edad es 15, no puedes hacer login


## Prioridad (salience)

Es un valor por defecto en 0, que indica la prioridad de la regla en relación a las demás. Cuanto mayor sea el valor numérico de la regla, antes será lanzada.

En el caso anterior en el que pedimos el nombre y la edad, vamos a darle mayor prioridad a una de las reglas, por ejemplo, **la regla del nombre**, para que siempre se nos solicite el nombre antes que la edad.

In [ ]:
from experta import *

EDAD_LIMITE = 18

class Registro(KnowledgeEngine):
  @DefFacts()
  def inicializacion(self):
    yield Fact(accion="inicio")

  @Rule(Fact(accion='inicio'),
  NOT(Fact(edad=W())), salience=0)
  def preguntar_edad(self):
    self.declare(Fact(edad=int(input("¿Cuál es tu edad? "))))

  @Rule(Fact(accion='inicio'),
  NOT(Fact(nombre=W())), salience=1)
  def preguntar_nombre(self):
    self.declare(Fact(nombre=input("¿Cuál es tu nombre? ")))

  @Rule(Fact(accion='inicio'),
  Fact(nombre=MATCH.nombre),
  Fact(edad=MATCH.edad & GE(EDAD_LIMITE)))
  def bienvenida(self, nombre, edad):
    print("¡Bienvenido %s! Tu edad es %s" % (nombre, edad))

  @Rule(Fact(accion='inicio'),
  Fact(nombre=MATCH.nombre),
  Fact(edad=MATCH.edad & LT(EDAD_LIMITE)))
  def no_bienvenida(self, nombre, edad):
    print("%s, tu edad es %s, no puedes hacer login" % (nombre, edad))

engine = Registro()
engine.reset() # Prepare the engine for the execution.
engine.run() # Run it!

¿Cuál es tu nombre? Pepe
¿Cuál es tu edad? 20
¡Bienvenido Pepe! Tu edad es 20


## Modificando Facts

Podemos realizar modificaciones en los hechos de la Base de Hechos aplicando los siguientes métodos:

- declare: para añadir nuevos Hechos a la Base de Hechos.
- retract: para eliminar Hechos de la Base de Hechos.
- modify: para modificar Hechos ya existentes en la Base de Hechos.
- duplicate: para añadir nuevos Hechos usando como plantilla Hechos que ya existan en la Base de Hechos.

In [ ]:
# Ejemplo del declare

m_inf = KnowledgeEngine()
m_inf.reset()
m_inf.declare(Fact(nombre='Pepe'))

m_inf.facts

FactList([(0, InitialFact()), (1, Fact(nombre='Pepe'))])

In [ ]:
# Ejemplo del retract

m_inf = KnowledgeEngine()
m_inf.reset()
m_inf.declare(Fact(nombre='Pepe'))
m_inf.declare(Fact(edad=20))

print('Hechos antes del retract: \n', m_inf.facts)

m_inf.retract(1)
print('Hechos después del retract: \n', m_inf.facts)

m_inf.declare(Fact(localidad='Cartagena'))
print('Hechos después del retract: \n', m_inf.facts)

Hechos antes del retract: 
 <f-0>: InitialFact()
<f-1>: Fact(nombre='Pepe')
<f-2>: Fact(edad=20)
Hechos después del retract: 
 <f-0>: InitialFact()
<f-2>: Fact(edad=20)
Hechos después del retract: 
 <f-0>: InitialFact()
<f-2>: Fact(edad=20)
<f-3>: Fact(localidad='Cartagena')


In [ ]:
# Ejemplo del retract usando Facts en reglas

class Test(KnowledgeEngine):
  @DefFacts()
  def start_data(self):
    yield Fact(color='blue')
    yield Fact(size='large')

  @Rule(Fact(size='small'),
        AS.f << Fact(size='large'))
  def now_is_small(self, f):
    self.retract(f)
    print('Now is small!')

In [ ]:
test = Test()
test.reset()
print(test.facts)
test.declare(Fact(size='small'))
test.run()
print(test.facts)

<f-0>: InitialFact()
<f-1>: Fact(color='blue')
<f-2>: Fact(size='large')
Now is small!
<f-0>: InitialFact()
<f-1>: Fact(color='blue')
<f-3>: Fact(size='small')


In [ ]:
# Ejemplo del modify

test = Test()
test.reset()

print('Hechos antes del modify: \n', test.facts)

test.modify(test.facts[1], color='red')

print('Hechos después del modify: \n', test.facts)

Hechos antes del modify: 
 <f-0>: InitialFact()
<f-1>: Fact(color='blue')
<f-2>: Fact(size='large')
Hechos después del modify: 
 <f-0>: InitialFact()
<f-2>: Fact(size='large')
<f-3>: Fact(color='red')


In [ ]:
# Ejemplo del modify usando Facts en reglas

class Test(KnowledgeEngine):
  @DefFacts()
  def start_data(self):
    yield Fact(color='blue')
    yield Fact(size='large')

  @Rule(Fact(size='small'),
        AS.f << Fact(size='large'))
  def change_size(self, f):
    self.modify(f, size='small')
    print('Now is small!')

In [ ]:
test = Test()
test.reset()
print(test.facts)
test.declare(Fact(size='small'))
test.run()
print(test.facts)

<f-0>: InitialFact()
<f-1>: Fact(color='blue')
<f-2>: Fact(size='large')
Now is small!
<f-0>: InitialFact()
<f-1>: Fact(color='blue')
<f-3>: Fact(size='small')


In [ ]:
# Ejemplo del duplicate

test = Test()
test.reset()

print('Hechos antes del duplicate: \n', test.facts)

test.duplicate(test.facts[1], texture='clean')

print('Hechos después del duplicate: \n', test.facts)

Hechos antes del duplicate: 
 <f-0>: InitialFact()
<f-1>: Fact(color='blue')
<f-2>: Fact(size='large')
Hechos después del duplicate: 
 <f-0>: InitialFact()
<f-1>: Fact(color='blue')
<f-2>: Fact(size='large')
<f-3>: Fact(color='blue', texture='clean')
